In [ ]:
import os

from easyCore.Fitting.Fitting import Fitter
from easyDiffractionLib import Phases
from easyDiffractionLib.interface import InterfaceFactory as Calculator
from easyDiffractionLib.Profiles.P1D import PDFParameters
from easyDiffractionLib.Jobs import Powder1DCW
from easyDiffractionLib.Interfaces.pdffit2 import readGRData

In [ ]:
# data_fname = os.path.realpath('examples\\PDF\\Ni-xray.gr')
data_fname = os.path.realpath('Ni-xray.gr')
data = readGRData(data_fname)

In [ ]:
cif_fname = os.path.realpath('Ni.cif')
phases = Phases.from_cif_file(cif_fname)

In [ ]:
calculator = Calculator()
calculator.switch("Pdffit2")

In [ ]:
parameters = PDFParameters()
job = Powder1DCW('Ni', parameters=parameters, phases=phases, interface=calculator)

fitter = Fitter(job, calculator.fit_func)

In [ ]:
parameters = job.parameters
parameters.qmax = 70
parameters.qdamp = 0.01
parameters.qbroad = 0.0
parameters.wavelength = 1.9122

pattern = job.pattern
pattern.zero_shift = 0.16
pattern.scale = 1.4473

In [ ]:
x_data = data[:, 0]

y_data = job.create_simulation(x_data)

In [ ]:
# plotting
# plotting
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
output_notebook()
FIGURE_WIDTH = 900
FIGURE_HEIGHT = 300

Gobs = data[:, 1]
Gfit = y_data

Gdiff = Gobs - Gfit
Gdiff_baseline = -10

Gdiff_show = Gdiff/5.0 + Gdiff_baseline

fig = figure()
fig.xaxis.axis_label = 'r (Å)'
fig.yaxis.axis_label = r"$$G (Å^{-2})\$$"
fig.title.text = 'Fit of nickel to x-ray experimental PDF'

fig.line(x_data, Gobs, legend_label='G(r) Data', color='steelblue', line_width=2)
fig.line(x_data, Gfit, legend_label='G(r) Fit', color='orangered', line_width=2)
fig.line(x_data, Gdiff_show, legend_label='G(r) Diff', color='grey', line_width=2)
show(fig)